<a href="https://colab.research.google.com/github/lhmin0614/CS492_Coursework1/blob/main/Q2_Incremental_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
%tensorflow_version 2.0x
import tensorflow as tf
from tensorflow import keras
from google.colab import drive
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import time
from sklearn.neighbors import KNeighborsClassifier

drive.mount('/content/drive')
mat = loadmat('/content/drive/MyDrive/CS492_teamProject/face.mat')

X = mat['X']
y = mat['l'][0]

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.0x`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
#train/test split
train_X1 = [i[[True, False, False, False, False]*104] for i in X] 
train_X2 = [i[[False, True, False, False, False]*104] for i in X]
train_X3 = [i[[False, False, True, False, False]*104] for i in X]
train_X4 = [i[[False, False, False, True, False]*104] for i in X]
test_X   = [i[[False, False, False, False, True]*104] for i in X]
train_y = y[[True, True, True, True, False]*104]
test_y  = y[[False, False, False, False, True]*104]

print("<<Data dimension>>")
print("train_X1~4 : ", len(train_X1)," * ", len(train_X1[0]))
print("test_X  : ", len(test_X)," * ",  len(test_X[0]))
print("train_y : ", len(train_y))
print("test_y  : ", len(test_y))

<<Data dimension>>
train_X1~4 :  2576  *  104
test_X  :  2576  *  104
train_y :  416
test_y  :  104


In [75]:
def pca(X, d):
  N = len(X)
  avg_X = [ sum(i)/len(i) for i in X ]   
  sub_X = [ list(np.asarray(x) - x_bar) for x, x_bar in zip(X, avg_X) ]
  A = np.matrix(sub_X)
  S = np.dot(A, A.T)
  eigvals, eigvecs_column = np.linalg.eigh(S)  
  P = eigvecs_column.T[-d:].T
  Lambda = np.diag(eigvals[-d:])

  return (np.array(avg_X), N, P, Lambda, S)

In [79]:
def IPCA(d):

  avg1, N1, P1, L1, S1 = pca(train_X1, d)

  for train_X in [train_X2, train_X3, train_X4]:
    avg2, N2, P2, L2, S2 = pca(train_X, d)
    
    #find combined mean
    N3 = N1+N2
    avg3 = (N1*avg1 + N2*avg2)/N2

    #find conbined matrix
    S3 = S1*N1/N3 + S2*N2/N3 + N1*N2*np.dot(np.matrix(avg1-avg2), np.matrix(avg1-avg2).T)/N3 

    #find Phi
    Phi = gs(np.concatenate((np.concatenate((P1,P2), axis=1),np.matrix(avg1-avg2).T), axis=1))

    #decomposition
    eigenproblem = np.dot(np.dot(Phi.T, S3),Phi)
    start = time.time()
    eigvals, R = np.linalg.eigh(eigenproblem)  
    stop = time.time()
    print(f"Training time: {stop - start}s")
    L3 = np.diag(eigvals)             # d1+d2+1 eigvals

    #prepare next step
    avg1 = avg3
    N1   = N3
    P1   = np.dot(Phi, R)
    L1   = L3
    S1   = S3
  
  return avg1, N1, P1, L1, S1


In [80]:
def projection(X, avg_X, P) :
  A = np.matrix([ list(np.asarray(x) - x_bar) for x, x_bar in zip(X, avg_X) ])
  W = np.dot(A.T, P)
  return W

In [82]:
d_cand = [10, 20, 30, 40]

for d_ in d_cand:
  print("\n\nIncremental PCA for "+ str(d_))
  avg_, N_, P_, L_, S_ = IPCA(d_)
  train_X  = [i[[True, True, True, True, False]*104] for i in X]
  prj_X  = projection(train_X, avg_, P_)
  validation_X = projection(test_X, avg_, P_)

  #kNN accuracy
  print("\nprint accuracy train / test")

  for i in range(7):
    classifier = KNeighborsClassifier(n_neighbors = i*2+1)
    classifier.fit(prj_X.tolist(), train_y)

    score_train = classifier.score(prj_X.tolist(), train_y)
    score_test = classifier.score(validation_X.tolist(), test_y)

    print("k="+str(i*2+1)+" : " +str(score_train) + "/" + str(score_test))
  
  #reconstruction error
  avg_X = [ sum(i)/len(i) for i in train_X ]

  sub_X = [ list(np.asarray(x) - x_bar) for x, x_bar in zip(train_X, avg_X) ]
  A = np.matrix(sub_X)
  W = np.dot(A.T, P_)
  X_weighted_sum = np.dot(W, P_.T)

  X_reconst = [list(np.asarray(x) + x_bar) for x, x_bar in zip(X_weighted_sum.tolist(), avg_X) ]

  reconst_err = [np.linalg.norm(i-j) for i, j in zip(X_reconst, np.asarray(np.matrix(train_X).T))]
  reconst_err_avg = sum(reconst_err)/len(reconst_err)

  print("\naverage reconstruction error is " + str(reconst_err_avg))



Incremental PCA for 10
Training time: 0.00036454200744628906s
Training time: 0.0005018711090087891s
Training time: 0.0008230209350585938s

print accuracy train / test
k=1 : 1.0/0.7019230769230769
k=3 : 0.6490384615384616/0.5769230769230769
k=5 : 0.6009615384615384/0.5288461538461539
k=7 : 0.5384615384615384/0.4519230769230769
k=9 : 0.4951923076923077/0.49038461538461536
k=11 : 0.43990384615384615/0.40384615384615385
k=13 : 0.40625/0.375

average reconstruction error is 1822.0884335968576


Incremental PCA for 20
Training time: 0.0007715225219726562s
Training time: 0.0012574195861816406s
Training time: 0.0018393993377685547s

print accuracy train / test
k=1 : 1.0/0.7307692307692307
k=3 : 0.6658653846153846/0.5480769230769231
k=5 : 0.6298076923076923/0.5192307692307693
k=7 : 0.5576923076923077/0.4519230769230769
k=9 : 0.4951923076923077/0.4807692307692308
k=11 : 0.4495192307692308/0.4230769230769231
k=13 : 0.42788461538461536/0.36538461538461536

average reconstruction error is 1719.16